In [1]:
%load_ext autoreload
%autoreload 
from DNC_predict import predict_label, pre_matt
import numpy as np
import glob

C:\Users\chint\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
def accuracy(pairs_nx3: np.ndarray, pairs_gt_mx3: np.ndarray):
    if len(pairs_gt_mx3)==0:
        print("Warning: no ground truth data!")
        return

    common_elements, common_idx1, common_idx2 = np.intersect1d(pairs_nx3[:, 0], pairs_gt_mx3[:, 0],
                                                               return_indices=True)

    pairs_col_2 = pairs_nx3[common_idx1, 1]
    pairs_gt_col_2 = pairs_gt_mx3[common_idx2, 1]
    idx = np.nonzero(pairs_col_2==pairs_gt_col_2)[0]
   # print(pairs_nx3[common_idx1][idx])
    return len(idx)/len(pairs_gt_mx3)


def ids_to_pairs(ids_1: np.ndarray, ids_2: np.ndarray) -> np.ndarray:
    """Generate ground truth of pairs from test_tracking data in https://osf.io/t7dzu/

    Notes
    -----
    The ids can be obtained from points[:, 3] loaded from the .npy files.
    The value -1 means the cell is not tracked. Other values (>=0) means the identified numeric id.
    """
    array1, array2 = ids_1.astype(int), ids_2.astype(int),
    filtered_array1 = array1[array1 != -1]
    filtered_array2 = array2[array2 != -1]

    # 找出相同元素及其在两个数组中的位置
    common_elements, common_idx1, common_idx2 = np.intersect1d(filtered_array1, filtered_array2, return_indices=True)

    # 对 array1, 2 进行排序，并记录原始索引
    sorted_indices_1 = np.argsort(array1)
    sorted_arr1 = array1[sorted_indices_1]
    sorted_indices_2 = np.argsort(array2)
    sorted_arr2 = array2[sorted_indices_2]

    # 使用 searchsorted 找到插入位置
    insert_positions_1 = np.searchsorted(sorted_arr1, common_elements)
    insert_positions_2 = np.searchsorted(sorted_arr2, common_elements)

    # 使用原始索引获取实际位置
    original_idx1 = sorted_indices_1[insert_positions_1]
    original_idx2 = sorted_indices_2[insert_positions_2]

    # 根据reference points的编号进行排序
    sorted_indices = np.argsort(original_idx1)
    pairs_gt = np.column_stack((original_idx1[sorted_indices], original_idx2[sorted_indices]))
    return pairs_gt

In [3]:
points1[:,3].astype(int), points2[:,3].astype(int)

NameError: name 'points1' is not defined

In [27]:
npy_files = glob.glob("../fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
i=3
points1 = np.load(npy_files[i])
points2 = np.load(npy_files[i+1])
pred_test_label, _ = predict_label(temp_pos=points1[:, :3], 
                              temp_label=np.arange(points1.shape[0]), 
                              test_pos=points2[:, :3], 
                              temp_color=None, cuda=False)
pairs_gt = ids_to_pairs(points1[:, 3],points2[:, 3])
pairs_pred = np.asarray([(i, id)for i, (id, p) in enumerate(pred_test_label) if p > 0])
# print(pairs_pred)
# print(pairs_gt)
accuracy(pairs_pred, pairs_gt)

0.07575757575757576

## The accuracy on a training data set

In [22]:
# the model can works with/without gpu(cuda=True/False)
accs = []
for i in range(1, 64):
    #print(f"i={i}")
    points1 = np.load(f"../fDNC_Data_PretrainedModel/Data/train_synthetic/syn_uns_300/syn_300_{i-1}.npy")
    points2 = np.load(f"../fDNC_Data_PretrainedModel/Data/train_synthetic/syn_uns_300/syn_300_{i}.npy")
    pred_test_label, _ = predict_label(temp_pos=points1[:, :3], 
                                  temp_label=np.arange(points1.shape[0]), 
                                  test_pos=points2[:, :3], 
                                  temp_color=None, cuda=False)
    pairs_gt = ids_to_pairs(points1[:, 3],points2[:, 3])
    pairs_pred = np.asarray([(i, id)for i, (id, p) in enumerate(pred_test_label) if p > 0])
    accs.append(accuracy(pairs_pred, pairs_gt))
print(f"average acc = {np.mean(accs)}, std={np.std(accs)}")

FileNotFoundError: [Errno 2] No such file or directory: '../fDNC_Data_PretrainedModel/Data/train_synthetic/syn_uns_300/syn_300_0.npy'

In [45]:
# the model can works with/without gpu(cuda=True/False)
accs = []
for i in range(1, 64):
    #print(f"i={i}")
    points1 = np.load(f"../fDNC_Data_PretrainedModel/Data/test_synthetic/syn_uns_200/syn_200_{i-1}.npy")
    points2 = np.load(f"../fDNC_Data_PretrainedModel/Data/test_synthetic/syn_uns_200/syn_200_{i}.npy")
    test_label, _ = predict_label(points1[:, :3], points1[:, 3], points2[:, :3], points2[:, 3], 
                                               None, cuda=False)
    accs.append(cal_acc(points2, test_label))
print(f"average acc = {np.mean(accs)}, std={np.std(accs)}")

average acc = 0.8007745292175136, std=0.07753937713942259


In [26]:
import glob
npy_files = glob.glob("../fDNC_Data/test_tracking/real_jeff_all" + "/*.npy")
accs = []
for i in range(100):
    print(f"{i}", end="\r")
    points1 = np.load(npy_files[i])
    points2 = np.load(npy_files[i+100])
    pred_test_label, _ = predict_label(temp_pos=points1[:, :3], 
                                  temp_label=np.arange(points1.shape[0]), 
                                  test_pos=points2[:, :3], 
                                  temp_color=None, cuda=False)
    pairs_gt = ids_to_pairs(points1[:, 3],points2[:, 3])
    pairs_pred = np.asarray([(i, id)for i, (id, p) in enumerate(pred_test_label) if p > 0])
    accs.append(accuracy(pairs_pred, pairs_gt))
print(f"average acc = {np.mean(accs)}, std={np.std(accs)}")

average acc = 0.11023646696460744, std=0.054240101576148336
